In [6]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from collections import Counter
stop_words = set(stopwords.words('english')) 
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [25]:
DATA = '/Users/jane/Desktop/search+youtube_v4/8_41.csv'

df = pd.read_csv(DATA)
data = df.dropna(subset=['query'])

data['query'].tolist()
    

    
            

['antwerpen centraal station internationaal loket',
 'Antwerpen-Centraal (Antwerp) train station - SNCB International',
 'antwerpen centraal station international office',
 'antwerpen centraal station map',
 'Map of Antwerp Central Station | The map of the station. | Flickr',
 'station map',
 'klm check in',
 'https://www.klm.com/travel/hu_en/prepare_for_travel/checkin_options/internet_checkin/',
 'huf 18 300.00 to eur',
 'klm thalys check in',
 'https://www.klm.com/travel/pl_en/plan_and_book/ticket_information/travel_by_train_or_bus_on_a_KLM_ticket/',
 'https://www.klm.com/travel/nl_nl/plan_and_book/ticket_information/travel_by_train_or_bus_on_a_klm_ticket/',
 'Find out more about travel from/to Belgium via Amsterdam Airport ...',
 'international ticket desk at Antwerp Central station',
 'ANTWERPEN-CENTRAAL - NMBS/SNCB',
 'Antwerp Central Railway Station | railcc',
 'Antwerpen-Centraal (Antwerp) train station - SNCB International',
 'Smooth KLM connections from Belgium to the rest of 

In [54]:
def tokenize_stem(df):
    searches = df['query'].tolist()
    tokens = []
    punct_list ={'-',':','&','|', 'https', '...', ','}
    for q in searches:
        sublist = word_tokenize(q)
        for token in sublist:
            if token not in stop_words | punct_list:
                tokens.append(token)
    return tokens

                  
                  
def top_ten(searchlist):
    return pd.Series(searchlist).value_counts().sort_values(ascending = False)

In [53]:
top_ten(tokenize_stem(data))

2020         357
flights      181
returning    147
Mar          139
LIM          121
            ... 
Nicki          1
Dahmer         1
dr             1
BIC            1
Quick          1
Length: 4962, dtype: int64